In [2]:
import requests
import pprint
import requests
from requests.exceptions import HTTPError
from datetime import datetime as dt
import pandas as pd

In [3]:
for url in ["https://api.trackico.io/v1/icos/all/?page=1&page_size=200000&q="]:
    try:
        response = requests.get(url)

        # If the response was successful, no Exception will be raised
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  # Python 3.6
    except Exception as err:
        print(f'Other error occurred: {err}')  # Python 3.6
    else:
        print('Success!')

Success!


# First pass

In [4]:
BTC_df = pd.DataFrame.from_dict(response.json()['results'])
BTC_df.head()

,category,country,ico_end,ico_start,id,logo_url,platform,pre_ico_end,pre_ico_start,profile_url,rating,short_description,slug,status,ticker,title,type
0,Trading,United Kingdom,2020-01-10T00:00:00Z,2019-10-25T00:00:00Z,9989,https://www.trackico.io/media/img/ico/dacx-dig...,Ethereum,None,None,https://www.trackico.io/ico/dacx/,5.0,Digital Asset & Commodity Exchange leverages a...,dacx,Ongoing,DACX,DACX,IEO
1,Other,Malta,2020-04-30T23:59:00Z,2020-01-01T12:00:00Z,9988,https://www.trackico.io/media/img/ico/gigajoul...,Ethereum,2019-12-31T00:00:00Z,2019-11-01T00:00:00Z,https://www.trackico.io/ico/gigajoule/,4.4,Gigajoule Floating Solutions is an electricity...,gigajoule,Pre-Sale,GIGJ,Gigajoule,STO
2,Travel,Switzerland,None,None,9986,https://www.trackico.io/media/img/ico/protrave...,Ethereum,2020-01-24T16:00:00Z,2019-12-13T16:00:00Z,https://www.trackico.io/ico/protravelcoin/,3.9,ProTravelcoin is a cryptocurrency designed to ...,protravelcoin,Upcoming,PTR,ProTravelcoin,ICO
3,Cryptocurrency,Malta,2019-11-28T08:00:00Z,2019-10-30T08:00:00Z,9978,https://www.trackico.io/media/img/ico/global-c...,Ethereum,None,None,https://www.trackico.io/ico/global-crypto-alli...,5.0,Creator of the very first ERC777 smart contrac...,global-crypto-alliance,Ongoing,CALL,Global Crypto Alliance,IEO
4,Platform,Singapore,2019-11-24T00:00:00Z,2019-10-10T00:00:00Z,9970,https://www.trackico.io/media/img/ico/statizex...,Ethereum,None,None,https://www.trackico.io/ico/statizex/,4.1,We defined our development strategy based on e...,statizex,Ongoing,STAT,Statizex,IEO


# Function that calls 4,000 cryptos (ICO and IEO) and  from TrackICO. All features included.

In [5]:
def get_Track_ICO_API():
    '''
    Function that calls 4,000 cryptos (ICO and IEO) and  from TrackICO. All features included.
    Tickers as index.
    Features = ['title', 'type', 'ticker', 'rating', 'status', 'country', 'platform', 'pre_ico_start', 'pre_ico_end', 'ico_start', 'ico_end']
    '''
    
    ICO_df = pd.DataFrame()

    for i in range(0,8):
        response = requests.get(f"https://api.trackico.io/v1/icos/all/?page=1&page_size=2000000&q=")

        result = pd.DataFrame.from_dict(response.json()['results'])

        ICO_df = pd.concat([result,ICO_df])
    
    ICO_df.drop(['slug','id','category','profile_url','logo_url','short_description'], axis=1, inplace=True)
    ICO_df.reset_index(drop=True, inplace = True)
    ICO_df.set_index('ticker', inplace = True)
    
    return ICO_df

In [6]:
print(get_Track_ICO_API().shape)

(4000, 10)


In [9]:
trackIPO_df = get_Track_ICO_API()

In [10]:
trackIPO_df.head()

,country,ico_end,ico_start,platform,pre_ico_end,pre_ico_start,rating,status,title,type
ticker,,,,,,,,,,
DACX,United Kingdom,2020-01-10T00:00:00Z,2019-10-25T00:00:00Z,Ethereum,None,None,5.0,Ongoing,DACX,IEO
GIGJ,Malta,2020-04-30T23:59:00Z,2020-01-01T12:00:00Z,Ethereum,2019-12-31T00:00:00Z,2019-11-01T00:00:00Z,4.4,Pre-Sale,Gigajoule,STO
PTR,Switzerland,None,None,Ethereum,2020-01-24T16:00:00Z,2019-12-13T16:00:00Z,3.9,Upcoming,ProTravelcoin,ICO
CALL,Malta,2019-11-28T08:00:00Z,2019-10-30T08:00:00Z,Ethereum,None,None,5.0,Ongoing,Global Crypto Alliance,IEO
STAT,Singapore,2019-11-24T00:00:00Z,2019-10-10T00:00:00Z,Ethereum,None,None,4.1,Ongoing,Statizex,IEO
